# 通过PyTorch手动实现ResNet

## ResNet网络结构

ResNet网络是参考了VGG19网络，在其基础上进行了修改，并通过短路机制加入了残差单元，如图所示。变化主要体现在ResNet直接使用stride=2的卷积做下采样，并且用global average pool层替换了全连接层。

ResNet的一个重要设计原则是：当feature map大小降低一半时，feature map的数量增加一倍，这保持了网络层的复杂度。

从图中可以看到，ResNet相比普通网络每两层间增加了短路机制，这就形成了残差学习，其中虚线表示feature map数量发生了改变。

图中展示的34-layer的ResNet，还可以构建更深的网络如表1所示。从表中可以看到，对于18-layer和34-layer的ResNet，其进行的两层间的残差学习，当网络更深时，其进行的是三层间的残差学习，三层卷积核分别是1x1，3x3和1x1，一个值得注意的是隐含层的feature map数量是比较小的，并且是输出feature map数量的1/4

<img src="./image/resnet.jpg"  alt="ResNet网络结构" />

图中展示的34-layer的ResNet，还可以构建更深的网络如下表所示。从表中可以看到，对于18-layer和34-layer的ResNet，其进行的两层间的残差学习，当网络更深时，其进行的是三层间的残差学习，三层卷积核分别是1x1，3x3和1x1，一个值得注意的是隐含层的feature map数量是比较小的，并且是输出feature map数量的1/4

<img src="./image/resnet_2.jpg"  alt="ResNet表格" />

下面我们再分析一下残差单元，ResNet使用两种残差单元，如下图所示。

左图对应的是浅层网络，而右图对应的是深层网络。

对于短路连接，当输入和输出维度一致时，可以直接将输入加到输出上。但是当维度不一致时（对应的是维度增加一倍），这就不能直接相加。

有两种策略：

（1）采用zero-padding增加维度，此时一般要先做一个downsamp，可以采用strde=2的pooling，这样不会增加参数；

（2）采用新的映射（projection shortcut），一般采用1x1的卷积，这样会增加参数，也会增加计算量。

短路连接除了直接使用恒等映射，当然都可以采用projection shortcut。

<img src="./image/resnet_3.png"  alt="ResNet shortcut" />

In [1]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F

from torchsummary import summary
from torchviz import make_dot, make_dot_from_trace

import os

In [2]:
# device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=0)

In [4]:
# hyper parameters
EPOCHS = 20
batch_size = 64
learning_rate = 0.001
classes = 10

In [5]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
])

In [6]:
# cifar10 32*32*3
train_dataset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=False, transform=transforms.ToTensor())

Using downloaded and verified file: ./cifar10_data\cifar-10-python.tar.gz
Extracting ./cifar10_data\cifar-10-python.tar.gz to ./cifar10_data


In [7]:
trainloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
len(trainloader)

782

In [9]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [10]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [11]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [12]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [18]:
test_net_50 = ResNet50().to(device)
y_50 = test_net_50(torch.randn(1, 3, 32, 32).to(device))
print(y_50.size())

torch.Size([1, 10])


In [19]:
summary(test_net_50, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]           4,096
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
            Conv2d-7          [-1, 256, 32, 32]          16,384
       BatchNorm2d-8          [-1, 256, 32, 32]             512
            Conv2d-9          [-1, 256, 32, 32]          16,384
      BatchNorm2d-10          [-1, 256, 32, 32]             512
       Bottleneck-11          [-1, 256, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]          16,384
      BatchNorm2d-13           [-1, 64, 32, 32]             128
           Conv2d-14           [-1, 64,

In [13]:
test_net_18 = ResNet18().to(device)
y = test_net_18(torch.randn(1, 3, 32, 32).to(device))
print(y.size())

torch.Size([1, 10])


In [14]:
summary(test_net_18, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [15]:
net18_model = ResNet18().to(device)

In [16]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net18_model.parameters(), lr=learning_rate)

In [17]:
# for updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [18]:
# train model
pre_epoch_total_step = len(trainloader)
current_lr = learning_rate
net18_model.train()
train_loss = 0
correct = 0
total = 0
for epoch in range(EPOCHS):
    for i, (x, y) in enumerate(trainloader):
        x = x.to(device)
        y = y.to(device)

        # forward
        prediction = net18_model(x)
        loss = criterion(prediction, y)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = prediction.max(1)
        total += y.size(0)
        correct += predicted.eq(y).sum().item()
        

        if (i+1) % 100 == 0:
            template = r"Epoch:{}/{}, step:{}/{}, Loss:{:.6f}, Acc: {:.6f}, (correct:{}/total:{})"
            print(template.format(epoch+1,
                                  EPOCHS, 
                                  i+1, 
                                  pre_epoch_total_step, 
                                  train_loss/(i+1),
                                  100.*correct/total,
                                 correct,
                                 total))

    # decay learning rate
    if (epoch+1) % 20 == 0:
        current_lr = current_lr/2
        update_lr(optimizer, current_lr)

Epoch:1/20, step:100/782, Loss:1.978853, Acc: 26.125000, (correct:1672/total:6400)
Epoch:1/20, step:200/782, Loss:1.855718, Acc: 30.453125, (correct:3898/total:12800)
Epoch:1/20, step:300/782, Loss:1.768279, Acc: 33.708333, (correct:6472/total:19200)
Epoch:1/20, step:400/782, Loss:1.693645, Acc: 36.714844, (correct:9399/total:25600)
Epoch:1/20, step:500/782, Loss:1.626807, Acc: 39.409375, (correct:12611/total:32000)
Epoch:1/20, step:600/782, Loss:1.571510, Acc: 41.812500, (correct:16056/total:38400)
Epoch:1/20, step:700/782, Loss:1.518394, Acc: 43.783482, (correct:19615/total:44800)
Epoch:2/20, step:100/782, Loss:12.717088, Acc: 46.849291, (correct:26423/total:56400)
Epoch:2/20, step:200/782, Loss:6.900116, Acc: 48.367834, (correct:30375/total:62800)
Epoch:2/20, step:300/782, Loss:4.951311, Acc: 49.696532, (correct:34390/total:69200)
Epoch:2/20, step:400/782, Loss:3.964879, Acc: 50.870370, (correct:38458/total:75600)
Epoch:2/20, step:500/782, Loss:3.370199, Acc: 51.953659, (correct:426

Epoch:14/20, step:400/782, Loss:14.846225, Acc: 80.115897, (correct:541263/total:675600)
Epoch:14/20, step:500/782, Loss:11.925484, Acc: 80.225220, (correct:547136/total:682000)
Epoch:14/20, step:600/782, Loss:9.978239, Acc: 80.333236, (correct:553014/total:688400)
Epoch:14/20, step:700/782, Loss:8.586854, Acc: 80.435809, (correct:558868/total:694800)
Epoch:15/20, step:100/782, Loss:60.538881, Acc: 80.624292, (correct:569530/total:706400)
Epoch:15/20, step:200/782, Loss:30.373648, Acc: 80.731762, (correct:575456/total:712800)
Epoch:15/20, step:300/782, Loss:20.325100, Acc: 80.831479, (correct:581340/total:719200)
Epoch:15/20, step:400/782, Loss:15.297526, Acc: 80.930402, (correct:587231/total:725600)
Epoch:15/20, step:500/782, Loss:12.282547, Acc: 81.028689, (correct:593130/total:732000)
Epoch:15/20, step:600/782, Loss:10.272424, Acc: 81.122291, (correct:599007/total:738400)
Epoch:15/20, step:700/782, Loss:8.836927, Acc: 81.218448, (correct:604915/total:744800)
Epoch:16/20, step:100/78

In [19]:
# test model
net18_model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for x, y in testloader:
        x = x.to(device)
        y = y.to(device)
        prediction = net18_model(x)
        _, predic = torch.max(prediction.data, dim=1)
        total += y.size(0)
        correct += (predic == y).sum().item()

    print("Accuracy:{}%".format(100 * correct / total))

model_path = r'./cifar10_data/model/'
if not os.path.exists(model_path):
    os.makedirs(model_path)
model_name = os.path.join(model_path, 'cifar10_resnet.ckpt')
# save model
torch.save(net18_model.state_dict(), model_name)

Accuracy:90.38%
